In [10]:
import json_lines
X=[]; y=[]; z=[]
with open ('reviews_291.jl', 'rb' ) as f:
  for item in json_lines.reader(f):
    X.append ( item [ 'text' ] )
    y.append ( item [ 'voted_up' ] )
    z.append ( item [ 'early_access' ] )

In [23]:
import pandas as pd
import numpy as np

text = pd.DataFrame(X)
votes = pd.DataFrame(y)
early = pd.DataFrame(z)
print(text.sample(5))
print(votes.sample())
print(early.sample())
text = text.to_numpy()


                                                      0
3462  ÐœÐµÑ€Ñ‚Ð²Ð°Ñ Ñ‚Ð¸ÑˆÐ¸Ð½Ð°, ÑÐµÑ€Ð²ÐµÑ€Ð° Ð¿...
2438  THIS IS â™¥â™¥â™¥â™¥â™¥â™¥â™¥ AWSOME :) MrBost...
4763                                          uma bosta
4683  For a buy2play game, I guess I expected more b...
98                                                    n
          0
4298  False
         0
251  False


In [41]:
subset_x = text[0:2000]

In [42]:
lang = []
for sentence in subset_x:
  lang.append(detect_language(sentence[0]))

In [44]:
lang_copy = lang

In [47]:
eng = []
for message in lang_copy:
  if message[1] == 'en':
    eng.append(message[0])

In [51]:
en_i = []
for i in range(len(lang_copy)):
  if lang_copy[i][1] == 'en':
    en_i.append(i)


In [62]:
import pickle
pickle.dump(eng, open("en_text.pkl", "wb"))

In [63]:
pickle.dump(en_i, open("en_i.pkl", "wb"))

In [69]:
a = pickle.load(open("en_i.pkl", "rb"))

In [52]:
len(en_i)

999

In [54]:
docs = eng

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None)
X = vectorizer.fit_transform(docs)

Upon first inspection it seems that the reviews consist of 
predominantly english sentences.   

There are also a considerable number of French, German, and Spanish reviews.   
And there are also reviews that solely contain symbols. Upon encoding these symbols 
to UTF-8, they turned out to be characters of the cyrillic alphabet.   


To predic the reviews, going to use bag of words.

Three main ways to use the data.

We can just make one big model with every single word included, even the 

An approach to be able to use all the data in the dataset and therefore predict any review would be
to pass the data through a google transalte API. Since for this exercise we don't care about the 
gramatical structure this method might be good enough.



In [33]:
sample_text = text[1][0]
print(text[1][0])

Most relaxing game ever!..


In [42]:
sample_text = text[11][0]
print(sample_text)
tokens = word_tokenize(sample_text)
stems = [stemmer.stem(token) for token in tokens]
print(stems)


i want this entire game shot directly into my veins
['i', 'want', 'thi', 'entir', 'game', 'shot', 'directli', 'into', 'my', 'vein']


In [20]:
import nltk
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer
tokenizer = CountVectorizer().build_tokenizer()
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize

print("sklearn:\t", tokenizer("Here's an example text, isn't it?"))
print("nltk whitespace:", WhitespaceTokenizer().tokenize("Here's an example text, isn't it?"))
print("nltk word:\t", word_tokenize("Here's an example text, isn't it?"))

print(tokenizer("likes liking liked"))
print(WhitespaceTokenizer().tokenize("likes liking liked"))
print(word_tokenize("likes liking liked"))

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
tokens = word_tokenize("Here's example text, isn't it?")
stems = [stemmer.stem(token) for token in tokens]
print(stems)
tokens = word_tokenize("likes liking liked")
stems = [stemmer.stem(token) for token in tokens]
print(stems)


sklearn:	 ['Here', 'an', 'example', 'text', 'isn', 'it']
nltk whitespace: ["Here's", 'an', 'example', 'text,', "isn't", 'it?']
nltk word:	 ['Here', "'s", 'an', 'example', 'text', ',', 'is', "n't", 'it', '?']
['likes', 'liking', 'liked']
['likes', 'liking', 'liked']
['likes', 'liking', 'liked']
['here', "'s", 'exampl', 'text', ',', 'is', "n't", 'it', '?']
['like', 'like', 'like']


[nltk_data] Downloading package punkt to /home/ek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
vectorizer = CountVectorizer()

# takes in an [ String ]
X = vectorizer.fit_transform(text[11])
print(vectorizer.get_feature_names())
print(X.toarray())


['directly', 'entire', 'game', 'into', 'my', 'shot', 'this', 'veins', 'want']
[[1 1 1 1 1 1 1 1 1]]


In [47]:
text_array = ["hello my name is", "howdy how are you!"]
vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(text_array)
print(vectorizer.get_feature_names())
print(X.toarray())


['are you', 'hello my', 'how are', 'howdy how', 'my name', 'name is']
[[0 1 0 0 1 1]
 [1 0 1 1 0 0]]


In [ ]:
nltk.download("stopwords")
vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words("english"))

In [52]:

docs = [
  'this is the first document', 'this is the second document', 'and the third one', 'Is this the first first document?'
]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None)
X = vectorizer.fit_transform(docs)
print(vectorizer.get_feature_names())
print(X.toarray())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0.         1.22314355 1.51082562 1.22314355 0.         0.
  1.         0.         1.22314355]
 [0.         1.22314355 0.         1.22314355 0.         1.91629073
  1.         0.         1.22314355]
 [1.91629073 0.         0.         0.         1.91629073 0.
  1.         1.91629073 0.        ]
 [0.         1.22314355 3.02165125 1.22314355 0.         0.
  1.         0.         1.22314355]]


In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# first 1000 articles from news dataset at https://www.kaggle.com/snapcrack/all−the−news
text = pd.read_csv(’articles1_1000.csv’)
text.head()
x = text[’content’]
vectorizer = TfidfVectorizer(stop_words = ’english’, max_df=0.2)
X = vectorizer.fit_transform(x)
indices = np.arange(x.from sklearn.model_selection import train_test_split
train, test = train_test_split(indices, test_size=0.2)

from sklearn.metrics.pairwise import cosine_distances
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=3,metric=cosine_distances).fit(X[train])

test=[test[0]]
found = nbrs.kneighbors(X[test], return_distance=False)

test_i=0
print(’text:\n%.300s’%x[test[test_i]])
for i in found[0]:
  print(’match %d:\n%.300s’%(i,x[train[i]]))

SyntaxError: invalid character in identifier (<ipython-input-53-287241588fda>, line 5)

In [40]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ek/College/MachineLearning/Final/translate-302719-4cc72fc518f9.json'
USER = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

def detect_language(text):
    """Detects the text's language."""
    from google.cloud import translate_v2 as translate
    translate_client = translate.Client()
    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.detect_language(text)
    return [text, result["language"], result["confidence"]]

In [38]:
a = detect_language("hello")

Text: hello
Confidence: 1
Language: en


In [39]:
print(a)

['hello', 'en', 1]
